In [1]:
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

#模型
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline


import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train_url="/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/train.csv"
train_raw=pd.read_csv(train_url)

test_url="/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/test.csv"
test_raw=pd.read_csv(test_url)

In [3]:
#拆分数据集
from sklearn.cross_validation import train_test_split  

#标注正样本和负样本
train_pos=train_raw[train_raw['target']==1]
print train_pos['target'].value_counts()
train_neg=train_raw[train_raw['target']==0]
print train_neg['target'].value_counts()

#df转narray
train_pos=np.array(train_pos)
train_neg=np.array(train_neg)
#数据集拆分
train_pos_new,validation_pos_new = train_test_split(train_pos,  
                                                   test_size = 0.2,  
                                                   random_state = 0)  
train_neg_new,validation_neg_new = train_test_split(train_neg,  
                                                   test_size = 0.2,  
                                                   random_state = 0)  
#数据集拼接
train_new=np.concatenate([train_pos_new,train_neg_new],axis=0) #在纵轴上合并

validation_new=np.concatenate([validation_pos_new,validation_neg_new],axis=0) #在纵轴上合并


#修改train、validation、test
train_temp=pd.DataFrame(train_new)
train_temp.columns=train_raw.columns
train=train_temp

validation_temp=pd.DataFrame(validation_new)
validation_temp.columns=train_raw.columns
validation=validation_temp

test=test_raw


1    21694
Name: target, dtype: int64
0    573518
Name: target, dtype: int64


In [4]:
train['tag']='train'
validation['tag']='validation'
test['tag']='test'

test.insert(1,'target',2)

print train.shape
print validation.shape
print test.shape

alldata=pd.concat([train,validation,test]) #在纵轴上合并
print alldata.shape

(476169, 60)
(119043, 60)
(892816, 60)
(1488028, 60)


In [5]:
#便于调试，做个备份
tmp_alldata=alldata

# 创建元数据dataframe

In [6]:
data = []
for f in train.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # Defining the data type 
    dtype = train[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [7]:
meta

,role,level,keep,dtype
varname,,,,
id,id,nominal,False,float64
target,target,binary,True,float64
ps_ind_01,input,interval,True,float64
ps_ind_02_cat,input,nominal,True,float64
ps_ind_03,input,interval,True,float64
ps_ind_04_cat,input,nominal,True,float64
ps_ind_05_cat,input,nominal,True,float64
ps_ind_06_bin,input,binary,True,float64
ps_ind_07_bin,input,binary,True,float64


In [8]:
alldata.columns

Index([u'id', u'target', u'ps_ind_01', u'ps_ind_02_cat', u'ps_ind_03',
       u'ps_ind_04_cat', u'ps_ind_05_cat', u'ps_ind_06_bin', u'ps_ind_07_bin',
       u'ps_ind_08_bin', u'ps_ind_09_bin', u'ps_ind_10_bin', u'ps_ind_11_bin',
       u'ps_ind_12_bin', u'ps_ind_13_bin', u'ps_ind_14', u'ps_ind_15',
       u'ps_ind_16_bin', u'ps_ind_17_bin', u'ps_ind_18_bin', u'ps_reg_01',
       u'ps_reg_02', u'ps_reg_03', u'ps_car_01_cat', u'ps_car_02_cat',
       u'ps_car_03_cat', u'ps_car_04_cat', u'ps_car_05_cat', u'ps_car_06_cat',
       u'ps_car_07_cat', u'ps_car_08_cat', u'ps_car_09_cat', u'ps_car_10_cat',
       u'ps_car_11_cat', u'ps_car_11', u'ps_car_12', u'ps_car_13',
       u'ps_car_14', u'ps_car_15', u'ps_calc_01', u'ps_calc_02', u'ps_calc_03',
       u'ps_calc_04', u'ps_calc_05', u'ps_calc_06', u'ps_calc_07',
       u'ps_calc_08', u'ps_calc_09', u'ps_calc_10', u'ps_calc_11',
       u'ps_calc_12', u'ps_calc_13', u'ps_calc_14', u'ps_calc_15_bin',
       u'ps_calc_16_bin', u'ps_calc_17_bin',

In [9]:
target='target'
IDcol = 'id'
tag='tag'
predictors = [x for x in alldata.columns if x not in [target, IDcol,tag]]

print '空值占比：'
line_count=alldata['target'].count()
for i in predictors :
    nan_cnt=  line_count - alldata[i][alldata[i] <> -1].count()
    if nan_cnt!=0:
        print i,": {:.5f}% ".format(nan_cnt/(line_count*1.0)*100 )

空值占比：
ps_ind_02_cat : 0.03515% 
ps_ind_04_cat : 0.01532% 
ps_ind_05_cat : 0.97572% 
ps_reg_03 : 18.10826% 
ps_car_01_cat : 0.01794% 
ps_car_02_cat : 0.00067% 
ps_car_03_cat : 69.09426% 
ps_car_05_cat : 44.81838% 
ps_car_07_cat : 1.93679% 
ps_car_09_cat : 0.09718% 
ps_car_11 : 0.00040% 
ps_car_12 : 0.00007% 
ps_car_14 : 7.15208% 


In [10]:
#剔除类别特征中空置率过高的特征，其他特征保留。保留原因参考（EDA中的分析内容）
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
alldata.drop(vars_to_drop, inplace=True, axis=1)
meta.loc[(vars_to_drop),'keep'] = False  # Updating the meta

In [11]:
#除了类别型特征，存在缺失的特征有 ps_reg_03、ps_car_11、ps_car_12、ps_car_14

#均值填充
mean_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
#最高频率填充
mode_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
#类别特征用-1表示，不做变化

alldata['ps_reg_03'] = mean_imp.fit_transform(alldata[['ps_reg_03']]).ravel()
alldata['ps_car_12'] = mean_imp.fit_transform(alldata[['ps_car_12']]).ravel()
alldata['ps_car_14'] = mean_imp.fit_transform(alldata[['ps_car_14']]).ravel()
alldata['ps_car_11'] = mode_imp.fit_transform(alldata[['ps_car_11']]).ravel()


In [12]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = alldata[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

Variable ps_ind_02_cat has 5 distinct values
Variable ps_ind_04_cat has 3 distinct values
Variable ps_ind_05_cat has 8 distinct values
Variable ps_car_01_cat has 13 distinct values
Variable ps_car_02_cat has 3 distinct values
Variable ps_car_04_cat has 10 distinct values
Variable ps_car_06_cat has 18 distinct values
Variable ps_car_07_cat has 3 distinct values
Variable ps_car_08_cat has 2 distinct values
Variable ps_car_09_cat has 6 distinct values
Variable ps_car_10_cat has 3 distinct values
Variable ps_car_11_cat has 104 distinct values


In [13]:
#ps_car_11_cat类别过多，转为对应target均值的形式

cat_perc = alldata[['ps_car_11_cat', 'target']].groupby(['ps_car_11_cat'],as_index=False).mean()
cat_perc.rename(columns={'target': 'ps_car_11_cat_tm'}, inplace=True)
alldata = pd.merge(alldata, cat_perc, how='inner', on='ps_car_11_cat')
alldata.drop('ps_car_11_cat', axis=1, inplace=True)
meta.loc['ps_car_11_cat','keep'] = False  # Updating the meta

## 创建哑变量

In [14]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
print('Before dummification we have {} variables in train'.format(alldata.shape[1]))
alldata = pd.get_dummies(alldata, columns=v, drop_first=True)
print('After dummification we have {} variables in train'.format(alldata.shape[1]))

Before dummification we have 58 variables in train
After dummification we have 110 variables in train


## 特征组合


In [15]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
interactions = pd.DataFrame(data=poly.fit_transform(alldata[v]), columns=poly.get_feature_names(v))
# Merge the interaction variables to the train data
print('Before creating interactions we have {} variables in train'.format(alldata.shape[1]))
alldata = pd.concat([alldata, pd.DataFrame(data=interactions, columns=poly.get_feature_names(v))], axis=1)
print('After creating interactions we have {} variables in train'.format(alldata.shape[1]))

Before creating interactions we have 110 variables in train
After creating interactions we have 487 variables in train


In [16]:
new_test=alldata[alldata['tag']=='test']

In [17]:
print new_test.shape

(892816, 487)


In [18]:
# new_train=alldata[alldata['tag']=='train']
# new_validation=alldata[alldata['tag']=='validation']
new_train_validation=alldata[ (alldata['tag']=='train') | (alldata['tag']=='validation')]
new_test=alldata[alldata['tag']=='test']


train_url="/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/train.csv"
train_raw=pd.read_csv(train_url)

new_train_validation.to_csv("/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/train_extends.csv")
new_test.to_csv("/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/test_extends.csv")
alldata.to_csv("/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/all_extends.csv")

In [20]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

# Regularized Greedy Forest
from rgf.sklearn import RGFClassifier     # https://github.com/fukatani/rgf_python


In [21]:
train=new_train_validation
test=new_test
# Preprocessing 
id_test = test['id'].values
target_train = train['target'].values

train = train.drop(['target','id','tag'], axis = 1)
test = test.drop(['target','id','tag'], axis = 1)

print(train.values.shape, test.values.shape)

((595212, 484), (892816, 484))


In [ ]:


# train = pd.read_csv('/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/input/train_extends.csv')
# test = pd.read_csv('/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/input/test_extends.csv')





class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res


        
# LightGBM params
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 650
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500
lgb_params['random_state'] = 99


lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.02
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['random_state'] = 99


lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['random_state'] = 99


# RandomForest params
rf_params = {}
rf_params['n_estimators'] = 200
rf_params['max_depth'] = 6
rf_params['min_samples_split'] = 70
rf_params['min_samples_leaf'] = 30


# ExtraTrees params
et_params = {}
et_params['n_estimators'] = 155
et_params['max_features'] = 0.3
et_params['max_depth'] = 6
et_params['min_samples_split'] = 40
et_params['min_samples_leaf'] = 18


# XGBoost params
xgb_params = {}
xgb_params['objective'] = 'binary:logistic'
xgb_params['learning_rate'] = 0.04
xgb_params['n_estimators'] = 490
xgb_params['max_depth'] = 4
xgb_params['subsample'] = 0.9
xgb_params['colsample_bytree'] = 0.9  
xgb_params['min_child_weight'] = 10



# XGBoost params,with scale_pos_weight
xgb_weight_params = {}
xgb_weight_params['objective'] = 'binary:logistic'
xgb_weight_params['learning_rate'] = 0.04
xgb_weight_params['n_estimators'] = 490
xgb_weight_params['max_depth'] = 4
xgb_weight_params['subsample'] = 0.9
xgb_weight_params['colsample_bytree'] = 0.9  
xgb_weight_params['min_child_weight'] = 4
xgb_weight_params['scale_pos_weight'] = 26


# CatBoost params
cat_params = {}
cat_params['iterations'] = 900
cat_params['depth'] = 8
cat_params['rsm'] = 0.95
cat_params['learning_rate'] = 0.03
cat_params['l2_leaf_reg'] = 3.5  
cat_params['border_count'] = 8
cat_params['gradient_iterations'] = 4


# Regularized Greedy Forest params
rgf_params = {}
rgf_params['max_leaf'] = 2000
rgf_params['learning_rate'] = 0.5
rgf_params['algorithm'] = "RGF_Sib"
rgf_params['test_interval'] = 100
rgf_params['min_samples_leaf'] = 3 
rgf_params['reg_depth'] = 1.0
rgf_params['l2'] = 0.5  
rgf_params['sl2'] = 0.005



lgb_model = LGBMClassifier(**lgb_params)

lgb_model2 = LGBMClassifier(**lgb_params2)

lgb_model3 = LGBMClassifier(**lgb_params3)

rf_model = RandomForestClassifier(**rf_params)

et_model = ExtraTreesClassifier(**et_params)
        
xgb_model = XGBClassifier(**xgb_params)

xgb_weight_model = XGBClassifier(**xgb_weight_params)

cat_model = CatBoostClassifier(**cat_params)

rgf_model = RGFClassifier(**rgf_params) 

gb_model = GradientBoostingClassifier(max_depth=5)

ada_model = AdaBoostClassifier()

log_model = LogisticRegression()


# Stacker score:         
stack = Ensemble(n_splits=3,
        stacker = log_model,
        base_models = (lgb_model,lgb_model2)) 


# Stacker score: 
# stack = Ensemble(n_splits=5,
#         stacker = log_model,
#         base_models = (lgb_model, xgb_model))


# Stacker score: 
# stack = Ensemble(n_splits=5,
#         stacker = log_model,
#         base_models = (lgb_model, xgb_weight_model))

        
y_pred = stack.fit_predict(train, target_train, test)        


Fit LGBMClassifier fold 1


In [ ]:



sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_pred
sub.to_csv('stacked_1.csv', index=False)